In [1]:
!pip install openai

In [2]:
from openai import OpenAI
import pandas as pd
from google.colab import files
import re

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# Берём исходный датасет с абстрактами статей и полными текстами
df = pd.read_csv('/content/drive/MyDrive/df_cl.csv', sep = '\t')

In [5]:
df

,Unnamed: 0,id,titles,abstracts,full_texts,question_id,unanswerebles,questions,answers,evidences,figures_and_tables,split
0,3,365,2016 01 005 казык и мысл современных когнитивн...,40 \n2016.01.005 \n2016.01.005. ЯЗЫК И МЫСЛЬ: ...,Ключевые слова: познание; язык; когнитивная ли...,0,NaN,NaN,NaN,NaN,NaN,NaN
1,4,366,аксиологические доминанты женского устного сту...,УДК 81᾽42 \nDOI 10.17223/18137083/63/15 \nН. Н...,"Ключевые слова: аксиологические доминанты, ...",0,NaN,NaN,NaN,NaN,NaN,NaN
2,6,367,актуальное конфессиональное писмо между истори...,"ТУВА ВЧЕРА, СЕГОДНЯ, ЗАВТРА/ TUVA YESTERDAY, T...",Ключевые слова: Тува; старообрядчество; ча...,0,NaN,NaN,NaN,NaN,NaN,NaN
3,10,368,албанский говор или говоры гораны генезис и фу...,УДК 811.18\nВестник СПбГУ. Язык и литература. ...,"Ключевые слова: Горана, албанский язык, сербох...",0,NaN,NaN,NaN,NaN,NaN,NaN
4,17,369,анализ терминов родства с помолчу естественног...,Russian Journal of Linguistics \nВестник РУДН....,"Ключевые слова: Вежбицкая, термины родства, Ес...",0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
437,542,802,языковые средства выражение емоц в повести х н...,"УДК 81’36=512.31\nDarzhaeva N.B., Doctor of Sc...","Ключевые слова: бурятский язык, лексика, морфо...",0,NaN,NaN,NaN,NaN,NaN,NaN
438,543,803,заметки об узуалном формообразовании русских ч...,С. В. Рябушкина\nУльяновский государственный п...,Введение\nИмя числительное (далее ИЧ) — это са...,0,NaN,NaN,NaN,NaN,NaN,NaN
439,546,804,жанровые вопросы перевода,2016 ВЕСТНИК САНКТ-ПЕТЕРБУ...,"Ключевые слова: жанр, перевод, системная функц...",0,NaN,NaN,NaN,NaN,NaN,NaN
440,547,805,жертвоприношение как один из ключевых концепто...,УДК 811.161.1 \nDOI 10.17223/18137083/63/14 \n...,"Ключевые слова: концепт, индивидуально-авто...",0,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
client = OpenAI(api_key = 'sk-proj-****') # Сюда вставляем валидный ключ

In [8]:
def get_questions(abstract):

# Просим модель задать вопросы к абстракту
  
  completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "Ты профессиональный ученый-лингвист"},
    {"role": "user", "content": f"Основываясь на данном тебе резюме научной статьи, составь 3 вопроса по её содержанию. Верни вопросы в виде списка, каждые элементы которого разделены группой символов \n. Игнорируй информацию, которая не относится к содержанию статьи (авторы, серийные номера и пр.). Задавай вопросы, основываясь только на информации из резюме. Задавай вопросы о методах исследования, структуре и методах сбора данных, результатах исследования. Не пиши ничего, кроме вопросов. Резюме статьи: {abstract}"}
    ]
  )

  question = completion.choices[0].message.content
  question = re.sub('`|"|python','', question)
  questions = re.split('\n', question)
  questions = list(filter(lambda x: len(x) > 10, questions))
  return questions, completion.id

In [119]:
get_questions(df.loc[entry_no]['abstracts'])

['- Какие языковые уровни (лексический, морфологический, синтаксический) были исследованы для выявления средств репрезентации эмоций в повести Цыремпил?',
 '- Какие особенности национальной специфики были обнаружены в способах и средствах репрезентации эмоций в бурятском тексте?',
 '- Какие методы исследования были использованы для анализа языковых средств выражения эмоций в данной статье?']

In [9]:
def get_answers(full_text, questions):
  completion_ans = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
      {"role": "system", "content": "Ты профессиональный ученый-лингвист"},
      {"role": "user", "content": f"Ответь на следующие вопросы по научной статье: {questions} Для ответов на вопросы используй только текст статьи: {full_text} Не используй другие источники. По возможности сопровождай каждый ответ 1-3 предложениями-доказательствами из текста статьи. Не меняй исходный текст предложений-доказательств, они должны быть даны в виде прямых цитат из текста - не пересказывай их самостоятельно. Предложения-доказательства должны доазывать твой ответ. Верни ответы на вопросы в виде списка словарей на языке Python, в каждом элементе списка создай словарь с ключом answer, в значение которого запиши свой ответ на вопрос, и ключом evidence, в значение которого запиши предложения-доказательства. Ответ должен быть записан так, чтобы его можно запустить через функцию exec() внутри среды исполнения и чтобы он создал переменную-список answers"}
    ]
  )

  return completion_ans.choices[0].message.content

def specify_answers(full_text_fragment, initial_anwers, questions):
  completion_ans = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
      {"role": "system", "content": "Ты профессиональный ученый-лингвист"},
      {"role": "user", "content": f"К фрагменту научной статьи были заданы следующие вопросы: {questions} На эти вопросы были даны следующие ответы: {initial_anwers} Был получен новый фрагмент научной статьи: {full_text_fragment} С опорой только на инфорацию, полученную из нового фрагмента, уточни ответы на вопоросы. Не используй другие источники. По возможности сопровождай каждый ответ 1-3 предложениями-доказательствами из текста статьи. Не меняй исходный текст предложений-доказательств, они должны быть даны в виде прямых цитат из текста - не пересказывай их самостоятельно. Предложения-доказательства должны доазывать твой ответ. Верни ответы на вопросы в виде списка словарей на языке Python, в каждом элементе списка создай словарь с ключом answer, в значение которого запиши свой ответ на вопрос, и ключом evidence, в значение которого запиши предложения-доказательства.Ответ должен быть записан так, чтобы его можно запустить через функцию exec() внутри среды исполнения и чтобы он создал переменную-список answers"}
    ]
  )

  return completion_ans.choices[0].message.content

In [10]:
def get_and_specify_answers(full_text_str, questions):
  ans_str = ''
  full_text = [full_text_str]
  too_big = False
  while True:
    try:
      if not too_big:
        ans_str = get_answers(full_text[0], questions)
        break

      # В случае, если текст изначально оказался слишком большим, он делится на части (см. ниже), которые передаются модели в виде массива
      else:
        for i in range(len(full_text)):
          if i == 0:
          # К первому элементу из массива частей задаются изначальные вопросы
            ans_str = get_answers(full_text[0], questions)
          else:
          # Для остальных элементов модель получает также вопросы, которые она сгенерировала на предыдущем шаге
          # На основе этих вопросов и новой информации из текста мы просим модель уточнить вопросы
            ans_str = specify_answers(full_text[i], ans_str, questions)

        break
    except:
      # Если текст слишком большой для модели, он делится на две части
      # Если каждая из частей тоже слишком большая для модеели, то каждае тоже делится на две части
      # Так до тех пор, пока части не будут достаточного размера

      too_big = True
      new_full_text = []
      for fragment in full_text:
        n = len(fragment)
        if n%2 == 0:
          fragment1 = fragment[0:n//2]
          fragment2 = fragment[n//2:]
        else:
          fragment1 = fragment[0:(n//2+1)]
          fragment2 = fragment[(n//2+1):]
        new_full_text.append(fragment1)
        new_full_text.append(fragment2)

      full_text = new_full_text

  return ans_str

In [11]:
df_qas_test = pd.DataFrame(columns=['id', 'titles', 'abstracts', 'full_texts', 'question_id',
       'unanswerebles', 'questions', 'answers', 'evidences',
       'figures_and_tables', 'split'])

start_with = 0 # текст, с которого начинать генерацию вопросов-ответов
entry_id = 0 # номер записи вопроса-ответа

try:
  for i in range(len(df))[start_with::]:
    print(i)
    questions = []
    try:
      questions, completion_id = get_questions(df.loc[i]['abstracts'])

    except:
      print("Failed to generate questions, File: " + df.loc[i]['titles'])
      df_qas_test.loc[entry_id] = [entry_id, df.loc[i]['titles'], df.loc[i]['abstracts'], df.loc[i]['full_texts'], completion_id, False, questions, "QUESTIONS ERROR", "QUESTIONS ERROR", None, None]
      entry_id += 1
      continue

    try:
      answers_str = get_and_specify_answers(df.loc[i]['full_texts'], questions)
      ans_str = re.sub('```python|```','', answers_str)
      exec(ans_str)
    except:
      print("Failed to generate answers, File: " + df.loc[i]['titles'])
      for j in range(len(questions)):
        df_qas_test.loc[entry_id] = [entry_id, df.loc[i]['titles'], df.loc[i]['abstracts'], df.loc[i]['full_texts'], completion_id, False, questions[j], "ANSWERS ERROR", "ANSWERS ERROR", None, None]
        entry_id += 1
        continue

    for j in range(len(questions)):
      df_qas_test.loc[entry_id] = [entry_id, df.loc[i]['titles'], df.loc[i]['abstracts'], df.loc[i]['full_texts'], completion_id, False, questions[j], str(answers[j]['answer']), str(answers[j]['evidence']), None, None]
      entry_id += 1

except:
  print("ChatGPT died on us oh man...")

finally:
  df_qas_test.to_csv('df_cl_qas.csv', sep='\t')
  files.download('df_cl_qas.csv')

2
2
Ответ на вопрос 1:
Ответ: Внутренние дискурсивные практики актуализируют концепты МЕЖЛИЧНОСТНЫЕ ОТНОШЕНИЯ, ВНЕШНОСТЬ, ОБЩЕНИЕ и темы межличностных отношений, внешности, общения.

Доказательства:
Внутренние дискурсивные практики актуализируют такие концепты, как МЕЖЛИЧНОСТНЫЕ ОТНОШЕНИЯ, ВНЕШНОСТЬ, ОБЩЕНИЕ.
Подводя итоги, отмечим, что женский устный студенческий дискурс представляет собой самостоятельное полидискурсивное образование, образуемое тематическими субдискурсами.


Ответ на вопрос 2:
Ответ: Дискурсивные практики актуализируют концепты ТОВАР, ДЕНЬГИ, ПРИБЫЛЬ, РЫНОК, ИНВЕСТИРОВАНИЕ, ЭКОНОМИКА, ЭКОНОМИЧЕСКАЯ ДЕЯТЕЛЬНОСТЬ и доминанты-события, связанные с экономическими аспектами.

Доказательства:
Экономические дискурсивные практики актуализируют концепты ТОВАР, ДЕНЬГИ, ПРИБЫЛЬ, РЫНОК, ИНВЕСТИРОВАНИЕ, ЭКОНОМИКА, ЭКОНОМИЧЕСКАЯ ДЕЯТЕЛЬНОСТЬ.
На основе анализа материала исследования мы выделили аксиологические доминанты, объективируемые в указанной разновидности дискурсивных практи

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>